In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models, layers
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
def create_model(n_params, 
                 dropout, 
                 recurrent_dropout, 
                 n_convlstm_layers = 2,
                 hidden_activation =  tf.keras.activations.tanh, 
                 optimizer = keras.optimizers.Adam()):

    # input layer
    input_layer = layers.Input(shape= (None,5,5,1) )
    
    # lstm layers
    lstm = input_layer
    for i in range( n_convlstm_layers ):
        lstm =  layers.ConvLSTM2D( 
            kernel_size= (1,1), 
            filters=n_params, 
            data_format= 'channels_last', 
            return_sequences = i<n_convlstm_layers-1,
            activation=hidden_activation,
            padding = "same",
            dropout=dropout, 
            recurrent_dropout=recurrent_dropout
        )( lstm )
        lstm = layers.BatchNormalization()(lstm)    

    output = layers.Conv2D(
        filters=1, kernel_size=(1, 1), activation="linear", padding="same"
    )( lstm )
    output_layer = layers.Reshape((5,5))(output)

    # compile
    model = models.Model( input_layer, output_layer )
    model.compile(
        loss= "MAE",
        optimizer=optimizer, 
    ) 
    
    print(model.summary())
    return model

def train_model(model, 
                x_train, 
                y_train,
                verbose = True, 
                save : "dir" = False,
                training_kwarg_overwrites : "dict" = {} ):
    
    # train until we run out of improvement
    callbacks = [
        keras.callbacks.ReduceLROnPlateau(monitor="val_loss", patience=5),
        keras.callbacks.EarlyStopping(monitor="val_loss", patience=15),
    ]
    
    # train model
    training_kwargs = {
        "x" : x_train,
        "y" : y_train, #dataset["train"]["y_scaled"],
        "epochs" : 200,
        "batch_size" : 64,
        "verbose" : verbose,
        "validation_split" : 0.2,
        "callbacks" : callbacks,
    } 
    training_kwargs.update(training_kwarg_overwrites)
    train_hist = model.fit( **training_kwargs )
    
    
    if save:
        Path(save).mkdir(parents=True, exist_ok=True) # make a home for the models
        train_start, train_end = [ f( dataset["dates"]["train"] ) for f in (min,max) ]
        model_name = "-".join( date.strftime("%Y%m%d") for date in [train_start, train_end] )
        model.save( save+model_name )
        
    return model, train_hist

model = create_model(n_params=10,dropout=0.1,recurrent_dropout=0.1,n_convlstm_layers=2)


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, 5, 5, 1)]   0         
                                                                 
 conv_lstm2d (ConvLSTM2D)    (None, None, 5, 5, 10)    480       
                                                                 
 batch_normalization (BatchN  (None, None, 5, 5, 10)   40        
 ormalization)                                                   
                                                                 
 conv_lstm2d_1 (ConvLSTM2D)  (None, 5, 5, 10)          840       
                                                                 
 batch_normalization_1 (Batc  (None, 5, 5, 10)         40        
 hNormalization)                                                 
                                                                 
 conv2d (Conv2D)             (None, 5, 5, 1)           11    

In [ ]:
# Load the MNIST dataset
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

# Normalize the images to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

# Reshape images to have a single channel (grayscale) and to a flat vector
train_images = train_images.reshape((train_images.shape[0], 28, 28, 1))
test_images = test_images.reshape((test_images.shape[0], 28, 28, 1))

# Build the model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(train_images, train_labels, epochs=5)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f'Test accuracy: {test_acc}')

# Make predictions
predictions = model.predict(test_images)

Epoch 1/5
1875/1875 [==============================] - 97s 51ms/step - loss: 0.1493 - accuracy: 0.9539
Epoch 2/5
1875/1875 [==============================] - 86s 46ms/step - loss: 0.0459 - accuracy: 0.9853
Epoch 3/5
1875/1875 [==============================] - 87s 46ms/step - loss: 0.0326 - accuracy: 0.9897
Epoch 4/5
1875/1875 [==============================] - 86s 46ms/step - loss: 0.0261 - accuracy: 0.9923
Epoch 5/5
313/313 [==============================] - 3s 7ms/step - loss: 0.0294 - accuracy: 0.9899
Test accuracy: 0.9898999929428101
313/313 [==============================] - 2s 7ms/step


: 

In [ ]:
# Show the first image and its predicted label
plt.imshow(test_images[0].reshape(28, 28), cmap='gray')
plt.title(f'Predicted label: {np.argmax(predictions[0])}')
plt.show()